# KMeans

KMeans is machine-learning algorithm (NP-hard), popularly employed for cluster analysis in data mining, and interesting for benchmarking and performance evaluation. 

The objective of the Kmeans algorithm to group a set of multidimensional points into a predefined number of clusters, in which each point belongs to the closest cluster (with the nearest mean distance), in an iterative process.


In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
ipycompss.start(monitor=2000)

In [ ]:
from pycompss.api.task import task

In [ ]:
import numpy as np

In [ ]:
def init_random(numV, dim, seed):
    np.random.seed(seed)
    c = [np.random.uniform(-3.5, 3.5, dim)]
    while len(c) < numV:
        p = np.random.uniform(-3.5, 3.5, dim)
        distance = [np.linalg.norm(p-i) for i in c]
        if min(distance) > 2:
            c.append(p)
    return c

In [ ]:
#@task(returns=list)  # Not a task for plotting
def genFragment(numV, K, c, dim, mode='gauss'):
    if mode == "gauss":
        n = int(float(numV) / K)
        r = numV % K
        data = []
        for k in range(K):
            s = np.random.uniform(0.05, 0.75)
            for i in range(n+r):
                d = np.array([np.random.normal(c[k][j], s) for j in range(dim)])
                data.append(d)
        return np.array(data)[:numV]
    else:
        return [np.random.random(dim) for _ in range(numV)]

In [ ]:
@task(returns=dict)
def cluster_points_partial(XP, mu, ind):
    dic = {}
    for x in enumerate(XP):
        bestmukey = min([(i[0], np.linalg.norm(x[1] - mu[i[0]])) for i in enumerate(mu)], key=lambda t: t[1])[0]
        if bestmukey not in dic:
            dic[bestmukey] = [x[0] + ind]
        else:
            dic[bestmukey].append(x[0] + ind)
    return dic

In [ ]:
@task(returns=dict)
def partial_sum(XP, clusters, ind):
    p = [(i, [(XP[j - ind]) for j in clusters[i]]) for i in clusters]
    dic = {}
    for i, l in p:
        dic[i] = (len(l), np.sum(l, axis=0))
    return dic

In [ ]:
@task(returns=dict, priority=True)
def reduceCentersTask(a, b):
    for key in b:
        if key not in a:
            a[key] = b[key]
        else:
            a[key] = (a[key][0] + b[key][0], a[key][1] + b[key][1])
    return a

In [ ]:
def mergeReduce(function, data):
    from collections import deque
    q = deque(list(range(len(data))))
    while len(q):
        x = q.popleft()
        if len(q):
            y = q.popleft()
            data[x] = function(data[x], data[y])
            q.append(x)
        else:
            return data[x]

In [ ]:
def has_converged(mu, oldmu, epsilon, iter, maxIterations):
    print("iter: " + str(iter))
    print("maxIterations: " + str(maxIterations))
    if oldmu != []:
        if iter < maxIterations:
            aux = [np.linalg.norm(oldmu[i] - mu[i]) for i in range(len(mu))]
            distancia = sum(aux)
            if distancia < epsilon * epsilon:
                print("Distance_T: " + str(distancia))
                return True
            else:
                print("Distance_F: " + str(distancia))
                return False
        else:
            # Reached the max amount of iterations
            return True

In [ ]:
def plotKMEANS(dim, mu, clusters, data):
    import pylab as plt
    colors = ['b','g','r','c','m','y','k']
    if dim == 2:
        from matplotlib.patches import Circle
        from matplotlib.collections import PatchCollection
        fig, ax = plt.subplots(figsize=(10,10))
        patches = []
        pcolors = []
        for i in range(len(clusters)):
            for key in clusters[i].keys():
                d = clusters[i][key]
                for j in d:
                    j = j - i * len(data[0])
                    C = Circle((data[i][j][0], data[i][j][1]), .05)
                    pcolors.append(colors[key])
                    patches.append(C)
        collection = PatchCollection(patches)
        collection.set_facecolor(pcolors)
        ax.add_collection(collection)
        x, y = zip(*mu)
        plt.plot(x, y, '*', c='y', markersize=20)
        plt.autoscale(enable=True, axis='both', tight=False)
        plt.show()
    elif dim == 3:
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        for i in range(len(clusters)):
            for key in clusters[i].keys():
                d = clusters[i][key]
                for j in d:
                    j = j - i * len(data[0])
                    ax.scatter(data[i][j][0], data[i][j][1], data[i][j][2], 'o', c=colors[key])
        x, y, z = zip(*mu)
        for i in range(len(mu)):
            ax.scatter(x[i], y[i], z[i], s=80, c='y', marker='D')
        plt.show()
    else:
        print("No representable dim")

## MAIN

Parameters (that can be configured in the following cell):
* numV: number of vectors (default: 10.000)                                                           
* dim: dimension of the points (default: 2)
* k: number of centers (default: 4)
* numFrag: number of fragments (default: 16)
* epsilon: convergence condition (default: 1e-10)
* maxIterations: Maximum number of iterations (default: 20)


In [ ]:
from pycompss.api.api import compss_wait_on
import time
%matplotlib inline

numV = 10000       # Vectors ----- with 1000 it is feasible to see the evolution across iterations
dim = 2            # Dimensions
k = 4              # Centers
numFrag = 16       # Fragments
epsilon = 1e-10    # Convergence condition
maxIterations = 20 # Max number of iterations

size = int(numV / numFrag)
startTime = time.time()
cloudCenters = init_random(k, dim, 8) # centers to create data groups
X = [genFragment(size, k, cloudCenters, dim, mode='gauss') for _ in range(numFrag)]

mu = init_random(k, dim, 7) # First centers
oldmu = []
n = 0
startTime = time.time()
while not has_converged(mu, oldmu, epsilon, n, maxIterations):
    oldmu = mu
    clusters = [cluster_points_partial(X[f], mu, f * size) for f in range(numFrag)]
    partialResult = [partial_sum(X[f], clusters[f], f * size) for f in range(numFrag)]
    mu = mergeReduce(reduceCentersTask, partialResult)
    mu = compss_wait_on(mu)
    mu = [mu[c][1] / mu[c][0] for c in mu]
    while len(mu) < k:
        # Add new random center if one of the centers has no points.
        indP = np.random.randint(0, size)
        indF = np.random.randint(0, numFrag)
        mu.append(X[indF][indP])
    n += 1

clusters = compss_wait_on(clusters)
    
print("-----------------------------")
print("Kmeans Time {} (s)".format(time.time() - startTime))
print("-----------------------------")
print("Result:")
print("Iterations: ", n)
print("Centers: ", mu)

plotKMEANS(dim, mu, clusters, X)

In [ ]:
ipycompss.stop()